In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from mpl_toolkits import mplot3d

In [3]:
outdir = "output/freqresppolezero/"

## Interactive pole-zero selector

In [4]:
def add_polezero(pz, pt):
    # Check if the point has a y component.
    pt[1] = pt[1] if np.abs(pt[1]) > 0.01 else 0
    if pt[1] != 0:
        pz.append(pt[0] + pt[1] * 1j)
        pz.append(pt[0] - pt[1] * 1j)
    else:
        pz.append(pt[0])
    return pz

In [14]:
# Poles and zeros of the system
allpoles = []
allzeros = []
state_zmag = True

# Z plane pole-zero plot.
fig_pz = figure("z plane", figsize=(6, 6))
ax_pz = fig_pz.add_subplot(1, 1, 1)

# Magnitude of z transform
fig_zmag = figure("z transform magnitude", figsize=(6, 6))
ax_zmag = plt.axes(projection='3d')

# Frequency response
fig_fs = figure("Frequency Response", figsize=(8, 4))
ax_fs = fig_fs.add_subplot(1, 1, 1)

def onclick(event):
    global allpoles
    global allzeros
    # Act only when there is a single click.
    if event.dblclick is True:
        return
    
    # Act according to the state.
    if event.button == 1:
        allpoles = add_polezero(allpoles, [event.xdata, event.ydata])
        allzeros = add_polezero(allzeros, [0, 0])
    elif event.button == 3:
        allzeros = add_polezero(allzeros, [event.xdata, event.ydata])
    print(allpoles)
    print(allzeros)
    # Update plot
    updateplots()

def onkeyrelease(event):
    global allzeros
    global allpoles
    # Set current state.
    if event.key == "c":
        allpoles = []
        allzeros = []
    updateplots()
    

def onkeyrelease_zmag(event):
    global state_zmag
    # Set current state.
    if event.key == "c":
        state_zmag = not state_zmag
    updateplots()

def get_zmag(x, y):
    # Zero components.
    numcomps = [np.abs(x + y * 1j - _z) for _z in allzeros]
    num = np.ones(x.shape)
    for _n in numcomps:
        num *= _n
    # Pole components
    dencomps = [np.abs(x + y * 1j - _p) for _p in allpoles]
    den = np.ones(x.shape)
    for _d in dencomps:
        den *= _d
    _H = num / den
    return _H


def update_polezero_plot():
    _t = np.linspace(-np.pi, np.pi, 720)
    ax_pz.clear()
    ax_pz.plot(cos(_t), sin(_t), lw=1, color="k")
    ax_pz.axhline(0, lw=2, color="k", alpha=0.3)
    ax_pz.axvline(0, lw=2, color="k", alpha=0.3)
    ax_pz.set_xlim([-1.5, 1.5])
    ax_pz.set_ylim([-1.5, 1.5])
    ax_pz.get_xaxis().set_ticks([])
    ax_pz.get_yaxis().set_ticks([])
    ax_pz.spines['right'].set_visible(False)
    ax_pz.spines['top'].set_visible(False)
    ax_pz.spines['left'].set_visible(False)
    ax_pz.spines['bottom'].set_visible(False)
    ax_pz.set_title("z plane", fontsize=18)
    for _z in allzeros:
        ax_pz.plot(_z.real, _z.imag, 'o', color="k")
    for _p in allpoles:
        ax_pz.plot(_p.real, _p.imag, 'x', color="k")
    
    fig_pz.canvas.draw()
    fig_pz.canvas.flush_events()

    
def update_zmag_plot():
    _t = np.linspace(-np.pi, np.pi, 720)
    # Compute the magnitude of the z transform.
    if state_zmag is True:
        N = 120
        x = np.outer(np.linspace(-1.5, 1.5, N), np.ones(N))
        y = x.copy().T # transpose
    else:
        N = 60
        theta = np.linspace(-np.pi, np.pi, N)
        r = np.linspace(0, 1, N)
        r, theta = np.meshgrid(r, theta)
        x = r * np.cos(theta)
        y = r * np.sin(theta)
    
    Hmag = get_zmag(x, y)
    
    # Update the z transform magnitude
    ax_zmag.clear()
    ax_zmag.plot_surface(x, y, Hmag, alpha=0.8, linewidth=0,
                         antialiased=True)
    ax_zmag.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax_zmag.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax_zmag.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    # make the grid lines transparent
    ax_zmag.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    ax_zmag.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    ax_zmag.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    ax_zmag.set_xlabel(r'$\Re\, z$', fontsize=20)
    ax_zmag.set_ylabel(r'$\Im\, z$', fontsize=20)
    ax_zmag.set_zlabel(r'$\vert H(z) \vert$', fontsize=20)
    ax_zmag.set_xlim(-1.6, 1.6)
    ax_zmag.set_ylim(-1.6, 1.6)
    ax_zmag.set_zlim(1 - 0.01 * np.max(Hmag), 1 + 0.5 * np.max(Hmag))
    
    x = np.cos(_t)
    y = np.sin(_t)
    Hcirc = get_zmag(x, y)
    ax_zmag.plot3D(x, y, Hcirc, 'red', lw=3)
    
    fig_zmag.canvas.draw()
    fig_zmag.canvas.flush_events()


def update_freqresp_plot():
    # Frequency Response
    _t = np.linspace(-np.pi, np.pi, 720)
    x = np.cos(_t)
    y = np.sin(_t)
    Hcirc = get_zmag(x, y)
    ax_fs.clear()
    ax_fs.grid(color='0.8', linestyle='--', linewidth=0.5);
    ax_fs.plot(_t, Hcirc / np.max(Hcirc), color="k", lw=2)
    ax_fs.set_xlim(-np.pi, np.pi)
    ax_fs.set_xlabel(r"$\Omega$", fontsize=20)
    ax_fs.set_ylabel(r"$\vert H(\Omega)\vert$", fontsize=20)
    ax_fs.set_title("Frequency Response", fontsize=20)
    xticks(fontsize=18)
    yticks(fontsize=18)
    
    fig_fs.canvas.draw()
    fig_fs.canvas.flush_events()
    
    plt.tight_layout()

    
def updateplots():
    update_polezero_plot()
    update_zmag_plot()
    update_freqresp_plot()

updateplots()

# Handle events from the pole-zero plots
cid1 = fig_pz.canvas.mpl_connect('button_press_event', onclick)
cid2 = fig_pz.canvas.mpl_connect('key_release_event', onkeyrelease)

# Handle events from the z transform magnitude plot.
cid3 = fig_zmag.canvas.mpl_connect('key_release_event', onkeyrelease_zmag)

[0.15161290322580623]
[0]
[0.15161290322580623, 0.15131578947368407]
[0, 0]
[(0.14920634920634934-0.032146749001089825j), (0.14920634920634934+0.032146749001089825j)]
[0]
[0.14920634920634934]
[0]
[0.14920634920634934, 0.14920634920634934]
[0, 0]
[0.14920634920634934, 0.14920634920634934, (0.34031746031746035+0.6024791136941519j), (0.34031746031746035-0.6024791136941519j)]
[0, 0, 0]
[0.3210526315789477]
[0]
[0.3210526315789477, 0.32105263157894726]
[0, 0]
[]
[1.0]
[]
[1.0, -1.0052631578947366]
[(-0.0052631578947368585+0.7002457002457003j), (-0.0052631578947368585-0.7002457002457003j)]
[1.0, -1.0052631578947366, 0]
[]
[-1.0052631578947369]
[]
[-1.0052631578947369, 1.0000000000000004]
[(0.46842105263157885+0.6836609336609336j), (0.46842105263157885-0.6836609336609336j)]
[-1.0052631578947369, 1.0000000000000004, 0]
[]
[(-0.9947368421052631+0.014742014742014975j), (-0.9947368421052631-0.014742014742014975j)]
[]
[-1.0052631578947366]
[]
[-1.0052631578947366, 0.9947368421052629]
[(-0.3315789

In [12]:
a = c = np.sqrt(2) - 1
b = -2 * np.sqrt(2)

In [13]:
np.roots([a, b, c])

array([6.67869733, 0.1497298 ])